In [12]:
# Modeling Vanilla Interest Rate Swaps Using QuantLib
# Jose Jimenez
import QuantLib as ql
import matplotlib.pyplot as plt 
Computation_date = ql.Date(20, 10, 2015)
print(Computation_date)
ql.Settings.instance().evaluationDate = Computation_date

October 20th, 2015


In [22]:
# Here we construct the yield curve objects. 
# For simplicity, we will use flat curves for discounting and 3-month LIBOR.

# Risk free rate
Rf = 0.01

LIBOR_rate = 0.02

# Actual/365 (Fixed) day count convention.
day_count = ql.Actual365Fixed()

# The yieldTermStructure object provides an method which gives us the discount factor 
# for a particular date (QuantLib.Date object) or time in years (with 0 = evaluationDate). 
# This method is called discount.

# FlatForward(): Flat interest-rate curve
PV_discount_curve = ql.YieldTermStructureHandle(
    ql.FlatForward(Computation_date, Rf, day_count)
)

LIBOR_curve = ql.YieldTermStructureHandle(
    ql.FlatForward(Computation_date, LIBOR_rate, day_count)
)

LIBOR_3months_index  = ql.Euribor3M(LIBOR_curve)  
#LIBOR_3months_index = ql.USDLibor(ql.Period(3, ql.Months), LIBOR_curve)

In [14]:
# To construct the Swap instrument, we have to specify the fixed rate leg and 
# floating rate leg.


UKcalendar = ql.UnitedKingdom()


Settlement_date = UKcalendar.advance(Computation_date, 5, ql.Days)

# 10 year SWAP contract maturity
Maturity_date = UKcalendar.advance(Settlement_date, 10, ql.Years)

# Tenor: amount of time left until the SWAP contract expires.

Fixed_leg_tenor = ql.Period(6, ql.Months)
Fixed_schedule = ql.Schedule(Settlement_date, Maturity_date, 
                             Fixed_leg_tenor, UKcalendar,
                             ql.ModifiedFollowing, ql.ModifiedFollowing,
                             ql.DateGeneration.Forward, False)



Floating_leg_tenor = ql.Period(3, ql.Months)
Floating_schedule = ql.Schedule (Settlement_date, Maturity_date, 
                              Floating_leg_tenor, UKcalendar,
                              ql.ModifiedFollowing, ql.ModifiedFollowing,
                              ql.DateGeneration.Forward, False)

In [15]:
# We construct a VanillaSwap object by including the fixed and float leg schedules created above.

# Notional = 100 millions
Notional = 100000000

# Let the fixed leg pay 3% coupon semiannually.
Fixed_rate = 0.03
Fixed_leg_daycount = ql.Actual360()

# Let the floating leg pay 3-month LIBOR quarterly.
Floating_spread = 0.004
Floating_leg_daycount = ql.Actual360()

Interest_Rate_Swap = ql.VanillaSwap(ql.VanillaSwap.Payer, Notional, Fixed_schedule, 
               Fixed_rate, Fixed_leg_daycount, Floating_schedule,
               LIBOR_3months_index, Floating_spread, Floating_leg_daycount )

In [16]:
# We evaluate the swap using a discounting engine.

Swap_engine = ql.DiscountingSwapEngine(PV_discount_curve)
Interest_Rate_Swap.setPricingEngine(Swap_engine)

In [17]:
# We evaluate the swap using a discounting engine.
# The cashflows for the fixed and floating leg can be extracted from the ir_swap object. 
# The fixed leg cashflows are shown below:
for i, cf in enumerate(Interest_Rate_Swap.leg(0)):
    print ("%2d    %-18s  %10.2f"%(i+1, cf.date(), cf.amount()))

 1    April 27th, 2016    1525000.00
 2    October 27th, 2016  1525000.00
 3    April 27th, 2017    1516666.67
 4    October 27th, 2017  1525000.00
 5    April 27th, 2018    1516666.67
 6    October 29th, 2018  1541666.67
 7    April 29th, 2019    1516666.67
 8    October 28th, 2019  1516666.67
 9    April 27th, 2020    1516666.67
10    October 27th, 2020  1525000.00
11    April 27th, 2021    1516666.67
12    October 27th, 2021  1525000.00
13    April 27th, 2022    1516666.67
14    October 27th, 2022  1525000.00
15    April 27th, 2023    1516666.67
16    October 27th, 2023  1525000.00
17    April 29th, 2024    1541666.67
18    October 28th, 2024  1516666.67
19    April 28th, 2025    1516666.67
20    October 27th, 2025  1516666.67


In [18]:
# The floating leg cashflows are shown below:
for i, cf in enumerate(Interest_Rate_Swap.leg(1)):
    print ("%2d    %-18s  %10.2f"%(i+1, cf.date(), cf.amount()))

 1    January 27th, 2016   607604.58
 2    April 27th, 2016     600986.48
 3    July 27th, 2016      600986.48
 4    October 27th, 2016   607604.58
 5    January 27th, 2017   607604.58
 6    April 27th, 2017     594368.67
 7    July 27th, 2017      600986.48
 8    October 27th, 2017   607604.58
 9    January 29th, 2018   620841.70
10    April 27th, 2018     581133.97
11    July 27th, 2018      600986.48
12    October 29th, 2018   620841.70
13    January 28th, 2019   600986.48
14    April 29th, 2019     600986.48
15    July 29th, 2019      600986.48
16    October 28th, 2019   600986.48
17    January 27th, 2020   600986.48
18    April 27th, 2020     600986.48
19    July 27th, 2020      600986.48
20    October 27th, 2020   607604.58
21    January 27th, 2021   607604.58
22    April 27th, 2021     594368.67
23    July 27th, 2021      600986.48
24    October 27th, 2021   607604.58
25    January 27th, 2022   607604.58
26    April 27th, 2022     594368.67
27    July 27th, 2022      600986.48
2

In [19]:
# Some other analytics such as the fair value, fair spread etc can be extracted as shown below.

print ("%-20s: %20.3f" % ("Net Present Value", Interest_Rate_Swap.NPV()))
print ("%-20s: %20.3f" % ("Fair Spread", Interest_Rate_Swap.fairSpread()))
print ("%-20s: %20.3f" % ("Fair Rate", Interest_Rate_Swap.fairRate()))
print ("%-20s: %20.3f" % ("Fixed Leg BPS", Interest_Rate_Swap.fixedLegBPS()))
print ("%-20s: %20.3f" % ("Floating Leg BPS", Interest_Rate_Swap.floatingLegBPS()))

Net Present Value   :         -5965530.814
Fair Spread         :                0.010
Fair Rate           :                0.024
Fixed Leg BPS       :           -96299.809
Floating Leg BPS    :            96420.424
